
We show how to solve a capacitated facility location problem (CFLP) in Julia using [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing), which is a metaheuristic to approximate the global optimum in a large search space for an optimization problem.

Here we used a so called combined simmulated annealing [@qin2012combined], which we explain briefly later.


## Problem statement

A company must select a subset of potential facility locations to minimize total cost associated with opening facilities and servicing customers. Each costumer has a specific demand and each facility has a fixed opening costs, a specific capacity and service costs based on the distance to customers.
Moreover we do not want to open more than we certain number of facilities.

## Mathematical model formulation

In order to have a more mathematical problem description we give a formulation as a mixed interger problem next:

### Sets

- $J=\{1,\ldots,m\}$ set of potential facilities
- $C=\{1,\ldots,n\}$ set of customers

### Parameters

- $c^f_j\in\mathbb{R}⁺$ fix opening cost of facility $j$
- $q_j\in\mathbb{N}^+$ capacity of facility $j$
- $d_i\in\mathbb{N}^+$ demand of customer $i$
- $c^v_{i,j}\in\mathbb{R}^+$ servicing costs from facility $j$ to customer $i$

### Decision variables

- $x_{ij}$, real, demand of customer $i$ served by facility $j$
- $y_j$, binary, 1 if and only if facility $j$ is open

### Objective

$$
\min \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij}
$$

### Constraints

- (c1) Each client's demand is served:

$$
\sum_j x_{ij} = d_i, \; \forall i \in C
$$

- (c2) A facility can serve a client only if its open:

$$
x_{ij} \leq d_i y_j, \forall i\in C, \forall j \in J
$$

- (c3) A facility can not provide more than its capacity

$$
\sum_i x_{ij} \leq q_j y_j, \forall j \in J
$$

- (c4) There is maximal number of facilities allowed to be opened

$$
\sum_j y_j \leq k, 
$$

### Problem formulation as a Mixed Interger Linear Programm


$$
\begin{array}{lll}
\min & \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij} &\\
s.t. & \sum_j x_{ij} = d_i & \forall i \in C\\
     & x_{ij} \leq d_i y_j & \forall i\in C, \forall j \in J\\
     & \sum_i x_{ij} \leq q_j y_j & \forall j \in J \\
     & \sum_j y_j \leq k & \\
     & x_{ij}\geq0 & \\
     & y_j\in\{0,1\}&
\end{array}
$$

## Implementation

For our simulated annealing implementation we choosed a so called a combined simmulated annealing [@qin2012combined], which means we have to layers:

1. an outer layer algorithm (OLSA), which optimizes the facility location decision
2. an inner layer algorithm (ILSA), which optimizes the demand allocation decision based on the open facility decision given by the outer layer algorithm.

and in each layer an simumalted annealing is used.

But before we look at the implementation and apply it to some examples, lets briefly give you a brief overview:

1. Data Structures 
2. I/O and Tests
3. Additonal functions e.g. to generate initial solutions
5. Combined simmulated annealing implementation
8. Solve examples

In [1]:
using Random
using LinearAlgebra
using JLD2

In [2]:
# set seed for reporducibility
Random.seed!(123);
Random.TaskLocalRNG();

## CFLP data structure

We define two data Data Structures constisting of

   1. `cflp_data` to hold the problem data and
   2. `cflp_solution` to hold one solution, which does not necessarily need to

be optimal.

In [3]:
include("./SA_DataSturctures.jl")

## I/O and tests functions

For simplicity of this showcase we wrote a function

1. to randomly generate data (`gen_data`)
2. print the solution in a more human readable way (`print_solution`) and
3. test the solution for feasibility (`test_cflp_solution`).

In [4]:
include("./IO.jl")
include("./Tests.jl")

test_cflp_solution (generic function with 2 methods)

## Additional functions

1. generate an initial solution
2. generate a client to facility assigment based on an heuristic
3. generate candidates for inner and outer layer simulated annealing

In [5]:
include("./SA_Algorithms.jl")

SA_cflp (generic function with 5 methods)

## Combined simmulated annealing implementation

We will implement a so called a combined simmulated annealing [@qin2012combined], which means we have to layers:

1. an outer layer algorithm (OLSA), which optimizes the facility location decision (see `SA_cflp`)
2. an inner layer algorithm (ILSA), which optimizes the demand allocation decision based on the open facility decision given by the outer layer algorithm (see `SA_client_facility_assignment`).

Maybe its valuable to note that from an user point of view, there is only a "simulate annealing function" `SA_cflp` and the ILSA simply means running a simulated annealing to find a good solution for assigning client demand to facilities based on a fixed descision on open facilities from the OLSA.

### Outer layer algorithm - open facility decision

In the outer layer algorithm we explore the search space (after starting from an initial solution) by swaping the status of a randomly choosen facility (i.e. `idx = rand(1:data.m)` below), e.g. we swap the status of the choosen facility from close to open and vice versa. This is done by the function `generate_candidate_facility` below.

Afterwards we run the inner layer algorithm to explore the search space with respect to the descision on assigning client demand to open facility, which we explain in the next section.

Of course other strategies migth be more useful, just to give two examples:

- if the number of facilities is small, we could simply to an lexicographic search. 
- if the number of possible facilities `m` is large but the number of maximal open facilities `k` is small, one could swap a random number of facilities simultaneously or use a rolling window with variable lenght.

In [6]:
function generate_candidate_facility(data::cflp_data, solution::cflp_solution, k::Int)
    """
    returns cflp_solution in which the decision on open facilities is randomly modified
    the allocation of client demand to facilities is done using the heuristic assignment
    """
    # generate new candidate for open facilities by swaping the value at a random index
    candidate = copy(solution.open_facilities)
    idx = rand(1:data.m)
    candidate[idx] = !candidate[idx]
    candidate_solution_heu = cflp_solution(data, candidate, heuristic_client_facility_assignment(data, candidate))
    
    # swap an index if generated candidate is infeasible            
    while !test_cflp_solution(data, candidate_solution_heu, k)
        idx = rand(1:data.m)
        candidate[idx] = !candidate[idx]
        candidate_solution_heu = cflp_solution(data, candidate, heuristic_client_facility_assignment(data, candidate))
    end     

    return candidate_solution_heu
end

generate_candidate_facility (generic function with 1 method)

### Inner layer algorithm - client to facility assignment

In the inner layer algorithm we explore the search space with respect to the assignment of client demand to facilities.

Based on the decision on open facilities made by the outer layer algorithm the initial solution is given by an heuristic assignment, which simply said assigns to each client the cheapest available facility.

From there we start shifting a random amount of client demand to a facility with free capacity:

1. find facility $j_{free}$ with free capacity
2. choose a random client $i_{rand}$
3. choose a random facility $j_{rand}$ which serves demand of client $i_rand$
4. choose a random demand (which is neither greater than assigned client demand to $j_{rand}$ nor the free capacity of $j_{free}$
5. move this demand from facility  $j_{rand}$ to $j_{free}$

This is done in the function `generate_candidate_assignment`.

We note that this strategy in the current implementation can get stuck (see doc string of `generate_candidate_assignment`), which means the inner layer algorithm is running but is not exploring the search space.
However this problem can be solved by checking the sufficient conditions first, before starting the inner layer algorithm. To keep the example simple we did not implemented this solution. Another option would be for example to swap a suitable demand between clients.

In [7]:
function generate_candidate_assignment(data::cflp_data, solution::cflp_solution)
    """
    returns a cflp_solution where the open_facilities are equal to the input, but
    a random part of one assignment of a client demand to facilities is move to a facilitiy 
    with free capacity

    N.B. In the current implementation it is possible, that the assignment matrix does not change
    (this happens when j_free = j_rand, i.e. we take demand from the facilitiy but assign it back again)
    In particular the SA is not exploring the search space, as its stays at the solution until the descision on open facilities changes
    
    In particular this happens when J = {j_rand}
    In particular this happend with random data when k has to be choosen close to number of facilities to fulfill feasibility

    In the next iteration we could implement a another decision, for example
    "that we simply swap a suitable demand between clients under the assumption that both have different facilities" 
    """
    
    assignment = copy(solution.assignment)
    
    # determine free capacity as difference of available capacity minus assigned capacity
    free_capacity = [data.capacity[j] * solution.open_facilities[j] for j in 1:data.m]
    free_capacity -= [sum(solution.assignment[j,:]) for j in 1:data.m]
    
    # get random facility with free capacity
    j_free = rand([j for j in 1:data.m if free_capacity[j]!=0])
    
    # move assigned capacity randomly to j_free
    ## but therefore we need a client and a facility and a random capacity to move first
    i_rand = rand([i for i in 1:data.n])
    J = [j for j in 1:data.m if solution.assignment[j, i_rand] != 0]
    j_rand = rand(J) 

    cap_move = rand(1:min(free_capacity[j_free], solution.assignment[j_rand,i_rand]))
    
    # move capacity
    assignment[j_free,i_rand] += cap_move
    assignment[j_rand,i_rand] -= cap_move
    
    return cflp_solution(data, solution.open_facilities, assignment)
end

generate_candidate_assignment (generic function with 1 method)

We now know how to generate a new candidate assignemt with `generate_candidate_assignment`, so lets look at the implementation of the inner layer algorithm (`SA_client_facility_assignment`) next.

In [8]:
function SA_client_facility_assignment(data::cflp_data, solution::cflp_solution,
        start_temp::Float64 = 10.0, end_temp::Float64 = 1.0, alpha::Float64 = 0.5, max_iter::Int64 = 5)
    """
    performs an simulated annealing to find an assignment of client demand to open facilities

    returns cflp_solution with local minimal costs
    """
    
    # initial solution
    current_solution = solution
    best_solution = cflp_solution(data, solution.open_facilities, solution.assignment)

    # iterate
    temp = start_temp

    while temp > end_temp
        for _ in 1:max_iter
            # generate new candidate assigning client demand to open facilities
            candidate_solution = generate_candidate_assignment(data, current_solution)  

            if  candidate_solution.cost < best_solution.cost 
                best_solution = cflp_solution(data, candidate_solution.open_facilities, 
                    candidate_solution.assignment)
            end

            if candidate_solution.cost < current_solution.cost
                current_solution = candidate_solution
            else
                acceptance_prob = exp( (current_solution.cost - candidate_solution.cost) / temp )
                if rand() < acceptance_prob
                    current_solution = candidate_solution
                end
            end          
        end        
        temp *= alpha
    end
    
    return  best_solution
    
end

SA_client_facility_assignment (generic function with 5 methods)

Now we have all ingredients for our outer layer algorithm `SA_cflp`:

In [9]:
function SA_cflp(
        data::cflp_data, k::Int64, start_temp::Float64 = 10.0, end_temp::Float64 = 1.0, alpha::Float64 = 0.5, max_iter::Int64 = 100)
    """
    TBA
    """
    # generate initial solution 
    current_solution = generate_initial_solution(data, k)
    best_solution = current_solution

    # iterate
    temp = start_temp

    while temp > end_temp
        for _ in 1:max_iter
            # generate a new decision on open facilities
            candidate_solution_heu = generate_candidate_facility(data, current_solution, k)
            
            # run simulated annealing on the assignment
            candidate_solution = SA_client_facility_assignment(data, candidate_solution_heu)

            if  test_cflp_solution(data, candidate_solution, k) && candidate_solution.cost < best_solution.cost 
                best_solution = candidate_solution
            end

            if candidate_solution.cost < current_solution.cost
                current_solution = candidate_solution
            else
                acceptance_prob = exp( (current_solution.cost - candidate_solution.cost) / temp )
                if rand() < acceptance_prob
                    current_solution = candidate_solution
                end
            end
        end
        temp *= alpha
    end

    if !test_cflp_solution(data, best_solution,k)
        println("did not found a feasible solution")
    end
    
    return  best_solution
end

SA_cflp (generic function with 5 methods)

## Solve examples

Lets apply our implementation to some examples.

### A simple CFLP

The following problem data for a CFLP is taken from [Mathematical Optimization: Solving Problems using SCIP and Python](https://scipbook.readthedocs.io/en/latest/flp.html).

Its so simple that one can easily derive an optimal solution by hand and we recall that one optimal solution has a total cost of ~5370.0 and uses facilities [2, 3].

In [10]:
# data
demand = [80, 270, 250, 160, 180]
capacity = [500,500,500]
cost_fix = [1000.,1000.,1000.]
cost_var = [4. 6. 9.; 5. 4. 7.; 6. 3. 4.; 8. 5. 3.; 10. 8. 4.]

data = cflp_data(cost_fix, cost_var, capacity, demand);

In [11]:
k = 2
sol = SA_cflp(data, k)
print_solution(sol)

Total cost are:5610.0
Open facilities:[2, 3]
Assignment:
customer   1 gets       80.000000 from facility   2
customer   2 gets      270.000000 from facility   2
customer   3 gets      150.000000 from facility   2
customer   3 gets      100.000000 from facility   3
customer   4 gets      160.000000 from facility   3
customer   5 gets      180.000000 from facility   3


### Example 2

The second example is larger and we may don't want to solve by hand.
We don't go into details of the data and just remark, that it was randomly generated to test the implementation.

*We did not print all the assignment of customers to facilities* to increase readability , but if you are interested simply set `dont_print_all` to false.

In [12]:
using JLD2
d = load("../data/data.jld2")["data"]
dont_print_all = true
data = cflp_data(d["cost_fix"], d["cost_var"], d["capacity"], d["demand"])

k = 9
example2 = SA_cflp(data, k)
print_solution(example2, dont_print_all)

Total cost are:14278.269999999999
Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]
Assignment:
customer   1 gets       18.000000 from facility   9
customer   2 gets       21.000000 from facility  13
customer   3 gets       12.000000 from facility  11
customer   4 gets       14.000000 from facility   3
customer   5 gets       18.000000 from facility  13


We recall the optimal solution of the MILP:
 
    Total cost are:14277.130000000001
    Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]

### Example 3

We just showcase how to randomly generate data for a CFLP and solve the corresponding CFLP by our combined simulated annealing implementation.

In [13]:
data = gen_data()
k = data.m - 1
sol = SA_cflp(data, k)
print_solution(sol)

Total cost are:6084.43
Open facilities:[2, 3, 4, 8]
Assignment:
customer   1 gets       28.000000 from facility   8
customer   2 gets       15.000000 from facility   2
customer   3 gets       25.000000 from facility   2
customer   4 gets        7.000000 from facility   2
customer   5 gets       24.000000 from facility   8
customer   6 gets        5.000000 from facility   3
customer   7 gets        6.000000 from facility   8
customer   8 gets       21.000000 from facility   4
customer   9 gets       23.000000 from facility   2
customer  10 gets       23.000000 from facility   4
customer  11 gets        4.000000 from facility   3
customer  12 gets       26.000000 from facility   8
customer  13 gets       14.000000 from facility   3
customer  14 gets       13.000000 from facility   2
customer  15 gets       28.000000 from facility   2
